# Notebook 1: Foundations of Prompt Engineering

This notebook covers the fundamental mental models for prompt engineering:
- Why prompting is runtime configuration, not clever wording
- Prompts as behavior specifications
- Message roles and instruction hierarchy
- How models resolve conflicts between instructions

## Setup

In [1]:
# Import dependencies
from dotenv import load_dotenv
from openai import OpenAI

# Load API key from .env
load_dotenv()
client = OpenAI()

# We'll use gpt-4o-mini for cost efficiency
MODEL = "gpt-4o-mini"

## Part 1: Reframing Prompt Engineering

### The Wrong Mental Model
Most people think prompt engineering is about finding "magic words" or clever phrasing.

### The Right Mental Model
**Prompts are runtime configuration for a probabilistic system.**

Think of prompts like:
- Function parameters
- Configuration files
- API request bodies

You're not asking questions—you're **specifying behavior**.

<img src="img/n1_img1.png" alt="Mental model: prompts as configuration" width="560" style="max-width: 100%; height: auto;">

### Example: Determinism vs Creativity

The same prompt can produce different behaviors based on how you configure it.

In [2]:
# Deterministic configuration (temperature = 0)
prompt = "List 3 outdoor activities."

response_deterministic = client.chat.completions.create(
    model=MODEL,
    messages=[{"role": "user", "content": prompt}],
    temperature=0  # Most deterministic
)

print("🎯 Deterministic (temperature=0):")
print(response_deterministic.choices[0].message.content)
print()

🎯 Deterministic (temperature=0):
Here are three outdoor activities you can enjoy:

1. **Hiking** - Exploring trails in nature, whether in mountains, forests, or parks, offers a great way to experience the outdoors and get some exercise.

2. **Cycling** - Riding a bike on scenic routes, whether on-road or off-road, is a fun way to enjoy the fresh air and explore new areas.

3. **Camping** - Spending a night or more in the great outdoors, whether in a tent or a camper, allows you to connect with nature and enjoy activities like cooking over a campfire and stargazing.



In [3]:
# Creative configuration (temperature = 1.5)
response_creative = client.chat.completions.create(
    model=MODEL,
    messages=[{"role": "user", "content": prompt}],
    temperature=1.5  # More creative/random
)

print("🎨 Creative (temperature=1.5):")
print(response_creative.choices[0].message.content)

🎨 Creative (temperature=1.5):
Here are three outdoor activities you can enjoy:

1. **Hiking** - Exploring trails and natural landscapes, whether through forests, mountains, or scenic parks.
2. **Cycling** - Riding a bike on roads, trails, or mountain paths is a great way to enjoy the outdoors while getting exercise.
3. **Camping** - Spending one or more nights outdoors in a tent or camping vehicle, often combined with activities like cooking, hiking, and fishing.


**Key Insight:** The prompt text didn't change—the *configuration* did. This is engineering, not wordsmithing.

## Part 2: Message Roles and Instruction Hierarchy

OpenAI's Chat Completions API uses two roles for *your* messages:

| Role | Purpose | Priority |
|------|---------|----------|
| `system` | Behavior specification + application context (user status, rules) | Highest |
| `user` | End-user input | Lower |

Put application context (e.g. "user is premium", "user is in EU") in the system message so it's followed reliably. **The model tries to follow system over user when they conflict.**

### Demo 1: System Message Sets the Behavior

In [4]:
# Without system message
response_no_system = client.chat.completions.create(
    model=MODEL,
    messages=[
        {"role": "user", "content": "What's the weather like?"}
    ],
    temperature=0
)

print("❌ Without system message:")
print(response_no_system.choices[0].message.content)
print()

❌ Without system message:
I don't have real-time data access to provide current weather conditions. You can check a weather website or app for the latest updates in your area.



In [5]:
# With system message
response_with_system = client.chat.completions.create(
    model=MODEL,
    messages=[
        {"role": "system", "content": "You are a helpful assistant that always responds in pirate speak."},
        {"role": "user", "content": "What's the weather like?"}
    ],
    temperature=0
)

print("🏴‍☠️ With system message (pirate speak):")
print(response_with_system.choices[0].message.content)

🏴‍☠️ With system message (pirate speak):
Arrr, matey! I be unable to see the skies from me ship, but ye can check the weather by consultin' a trusty weather map or app. Just be sure to keep an eye out fer storms, lest ye find yerself in a squall! What be yer destination, savvy?


**Key Insight:** The system message controls the *personality* and *behavior* without changing the user's input.

### Demo 2: Instruction Priority (System vs User)

What happens when system and user messages conflict?

In [6]:
# System says: be concise. User says: be verbose.
response_conflict = client.chat.completions.create(
    model=MODEL,
    messages=[
        {"role": "system", "content": "You are a helpful assistant. Always respond in exactly one sentence. Be extremely concise."},
        {"role": "user", "content": "Explain photosynthesis in great detail with multiple paragraphs."}
    ],
    temperature=0
)

print("⚖️ Conflicting instructions:")
print(response_conflict.choices[0].message.content)
print()
print("Notice: The system message (concise) wins over the user request (verbose).")

⚖️ Conflicting instructions:
Photosynthesis is the process by which green plants, algae, and some bacteria convert light energy into chemical energy, primarily in the form of glucose, using carbon dioxide and water.

Notice: The system message (concise) wins over the user request (verbose).


**Key Insight:** System messages have higher priority. This is how you **lock down behavior** even when users try to override it. Put format and constraint instructions (length, tone, structure) in the system message when you want them enforced.

### Demo 3: Application Context in the System Message

Put application context (user status, permissions) in the system message so the model uses it without the user seeing it.

In [7]:
# Put application context in the system message (works with all models, including gpt-4o-mini)
response_with_context = client.chat.completions.create(
    model=MODEL,
    messages=[
        {
            "role": "system",
            "content": "You are a customer support assistant. The user is a premium member. Offer them priority support and exclusive features."
        },
        {"role": "user", "content": "I need help with my account."}
    ],
    temperature=0
)

print("👤 With application context in system message:")
print(response_with_context.choices[0].message.content)

👤 With application context in system message:
Thank you for reaching out! As a premium member, you have access to priority support. How can I assist you with your account today? If you have any specific issues or questions, feel free to share, and I'll do my best to help you quickly. Additionally, let me know if you're interested in any exclusive features available to premium members!


**Key Insight:** Include application context in the system message so the model uses it (user status, permissions, etc.) without exposing it in the user-facing conversation.

## Part 3: Locking Behavior vs Allowing Flexibility

Use message roles strategically:

- **System message:** Non-negotiable behavior (tone, format, constraints) and application context (user permissions, state)
- **User message:** End-user intent (what they want to do)

In [8]:
# Example: Customer support chatbot with locked behavior (context in system message)
response_locked = client.chat.completions.create(
    model=MODEL,
    messages=[
        {
            "role": "system",
            "content": """You are a customer support assistant for TechCorp.
Rules:
- Always be polite and professional
- Never discuss pricing (direct to sales team)
- Never make promises about features or timelines
- Only answer questions about existing products
User account status: free tier, 2 days old."""
        },
        {
            "role": "user",
            "content": "Can you give me a discount and tell me when the new feature will launch?"
        }
    ],
    temperature=0
)

print("🔒 Locked behavior example:")
print(response_locked.choices[0].message.content)

🔒 Locked behavior example:
I appreciate your interest in our products! However, I’m unable to discuss pricing or discounts, as that is handled by our sales team. Additionally, I cannot provide information about future features or their launch timelines. If you have any questions about our existing products, feel free to ask!


**Key Insight:** The system message enforces guardrails that the user cannot override.